In [ ]:
# YOU HAVE TO RUN THIS CELL
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("postgresql+psycopg2:///credit_risk")

df = pd.read_sql("SELECT * FROM accepted_loans_ml_training;", engine)

df.head()

,loan_id,borrower_id,is_default,loan_amnt,installment,dti,income,term_months,purpose
0,96668860,25130,0,3975.0,143.69,23.67,43000.0,36,major_purchase
1,118207599,61983,0,4800.0,168.81,11.37,80000.0,36,other
2,127053376,33038,0,8400.0,268.85,8.76,50000.0,36,credit_card
3,69307155,80398,0,16000.0,531.36,16.89,179000.0,36,credit_card
4,21700773,10317,0,10000.0,315.17,22.38,31000.0,36,debt_consolidation


In [28]:
# YOU HAVE TO RUN THIS CELL
 
income_cap = df["income"].quantile(0.99)
#apply income cap
df["income"] = df["income"].clip(upper=income_cap)
df["income"].describe()
(df["income"] == income_cap).sum() # 866 outliers

np.int64(866)

In [29]:
# YOU HAVE TO RUN THIS CELL

#drop id cols
ids = ["loan_id", "borrower_id"]

target = "is_default"
num = ["loan_amnt", "installment", "dti", "income", "term_months"]
cat = ["purpose"]

#define x and y
X = df[num + cat]
y = df[target]

X.head(), y.head()

(   loan_amnt  installment    dti    income  term_months             purpose
 0     3975.0       143.69  23.67   43000.0           36      major_purchase
 1     4800.0       168.81  11.37   80000.0           36               other
 2     8400.0       268.85   8.76   50000.0           36         credit_card
 3    16000.0       531.36  16.89  179000.0           36         credit_card
 4    10000.0       315.17  22.38   31000.0           36  debt_consolidation,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: is_default, dtype: int64)

In [31]:
# YOU HAVE TO RUN THIS CELL

#split data intro training and test sets -- w/ 20% of data to test and the rest to training

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

X_train.shape, X_test.shape

((65168, 6), (16292, 6))

In [33]:
# YOU HAVE TO RUN THIS CELL

#scale num features, one-hot-encode cats
# applies transformations

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ],
    remainder="drop"
)

In [34]:
# YOU HAVE TO RUN THIS CELL

#baseline model - logistic regression

#feed data into log reg model

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

log_reg_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)

In [ ]:
# YOU HAVE TO RUN THIS CELL

log_reg_model.fit(X_train, y_train) #check if it works!!

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
# YOU HAVE TO RUN THIS CELL

#eval log reg model

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

y_pred = log_reg_model.predict(X_test)
y_proba = log_reg_model.predict_proba(X_test)[:, 1]

print("Accuracy:     ", accuracy_score(y_test, y_pred)) #correctly predicts about 66% of loans overall
print("Precision (1):", precision_score(y_test, y_pred)) #low precision when predicting default (1) -- only right 18% of the time
print("Recall (1):   ", recall_score(y_test, y_pred)) #catches about 50% of actual defaults -- not that great
print("F1 (1):       ", f1_score(y_test, y_pred)) #not great -- misses a lot of defaults + low-precision
print("ROC-AUC:      ", roc_auc_score(y_test, y_proba)) #0.64 -- okayish

Accuracy:      0.6581144119813406
Precision (1): 0.1818670489159649
Recall (1):    0.5027241208519069
F1 (1):        0.26710526315789473
ROC-AUC:       0.6419456555561791


In [13]:
df["is_default"].value_counts()

is_default
0    71365
1    10095
Name: count, dtype: int64

In [14]:
id_cols = ["loan_id", "borrower_id"]
target_col = "is_default"

numeric_features = [
    "loan_amnt",
    "installment",
    "dti",
    "income",
    "term_months"
]

categorical_features = [
    "purpose"
]

In [15]:
df[target_col].isna().sum()

np.int64(0)

In [ ]:
df[numeric_features + categorical_features].isna().sum() # check missing vals
# NO MISSING VALS!!

loan_amnt      0
installment    0
dti            0
income         0
term_months    0
purpose        0
dtype: int64

In [21]:
#check income
# df["income"].describe()

# df[df["income"] > 500000].head()

(df["income"] > 200000).sum()
#(df["income"] > 300000).sum()
#(df["income"] > 400000).sum()


np.int64(1005)

In [ ]:
df[df["loan_to_value_ratio"].notna()].head()
df["loan_to_value_ratio"].notna().sum()
df[df["loan_purpose"].notna()].head()
df["loan_purpose"].notna().sum()

,loan_id,borrower_id,is_default,loan_amnt,installment,dti,loan_to_value_ratio,income,term_months,purpose,loan_purpose,derived_loan_product_type


In [10]:
pd.read_sql("SELECT COUNT(*) FROM Accepted_Loans;", engine)

,count
0,281877


In [11]:
pd.read_sql("SELECT COUNT(*) FROM Accepted_Loans WHERE loan_to_value_ratio IS NOT NULL;",engine)

,count
0,199803
